# Tutorial - A Basic Encoding Script

In this tutorial, you will learn how to create an encoding from scratch, using the Bitmovin APIs and the Python SDK that wraps them. We will explain the concepts and the terminology that we use.

This tutorial concentrates on taking a single source file, encoding it into a ladder of multiple renditions, and creating a manifest which can be played back by any modern video player on most devices and browsers.

# Understanding how the API is composed
![object model](http://demo.bitmovin.com/public/learning-labs/encoding/ObjectModel_ABR.png)

For a complete description of the Bitmovin data model, check our [Object Model documentation]( https://bitmovin.com/docs/encoding/tutorials/understanding-the-bitmovin-encoding-object-model).

# A Little Setup
Let's import the Bitmovin API Client into our python script, as well as other dependencies we will need.

In [44]:
import sys
sys.path.append('../libs')

import collections
import os
import time
import uuid

from bitmovin_api_sdk import BitmovinApi, BitmovinApiLogger
from bitmovin_api_sdk import HttpsInput, S3Output, AwsCloudRegion
from bitmovin_api_sdk import StreamInput, StreamSelectionMode
from bitmovin_api_sdk import ProfileH264
from bitmovin_api_sdk import H264VideoConfiguration, PresetConfiguration
from bitmovin_api_sdk import Encoding, CloudRegion
from bitmovin_api_sdk import MuxingStream, Fmp4Muxing, EncodingOutput, AclEntry, AclPermission
from bitmovin_api_sdk import Stream, Status
from bitmovin_api_sdk import DashManifestDefault, DashManifestDefaultVersion
from bitmovin_api_sdk import AacAudioConfiguration

from dotenv import load_dotenv
_ = load_dotenv()

For the purpose of this tutorial, we also import a few additional helpers. You won't need these in your own scripts

In [74]:
import config as cfg
import helpers
from IPython.display import display, IFrame
from vdom import p, div, b, a

We are being quite specific about the Bitmovin objects that we want to import, just to make things clear for this example...

## Secret sauce
The API key is what you need to authenticate with the Bitmovin API. You can find it in the dashboard in the [Account section](https://bitmovin.com/dashboard/account)

It is a __secret__, and should be treated as such. If someone else gets hold of your key, they can run encodings on your account (or your organisation accounts) and get information about previous ones.

In [46]:
cfg.API_KEY = os.getenv('API_KEY', "")

The Organization ID indicates what Bitmovin account you want to create and process your encodings in. Leave it empty if you are using your own account. If you belong to a multi-tenant organization, you need to get the organisation ID from the dashboard in the [Organization section](https://bitmovin.com/dashboard/organization/overview).

In [47]:
cfg.ORG_ID = os.getenv('ORG_ID', "")

For this learning lab we have created an S3 bucket and a (very limited) user we can all use.

In [48]:
cfg.S3_BUCKET_NAME = os.getenv('S3_BUCKET_NAME', "")
cfg.S3_ACCESS_KEY = os.getenv('S3_ACCESS_KEY', "")
cfg.S3_SECRET_KEY = os.getenv('S3_SECRET_KEY', "")

Finally, to prevent conflicts between all our encodings, let's add something unique to each of us.

In [49]:
cfg.MY_ID = os.getenv('MY_ID', "")

We'll quickly run some checks to make sure that your setup is ready to use. This is where we use these helpers. 

In [50]:
msg = helpers.validate_config()
base_output_path = helpers.build_output_path()

display(
    div(
        p(f"{msg}. Your output files will be added to subdirectory ", b(f"{base_output_path}")),
        style={"color": "green"}
    )
)

<div style="color: green"><p>Your configuration appears complete. Your output files will be added to subdirectory <b>outputs/fabre-49bdbc9d-c8fb-4ce7-881c-d5df402028a9</b></p></div>

In a production script, you won't need this (or rather, you'll need to do something that is suitable for your workflow)

# Configuring our Encoding
Now that the boring bits are behind us, we are (finally) ready to start the real work.

First, we need to instantiate the API client with our secrets. 

In [51]:
api = BitmovinApi(api_key=cfg.API_KEY,
                  tenant_org_id=cfg.ORG_ID)

<img src="img/step1.svg" alt="Input and Output" width="320px" align="right"/>

## Input and Output locations
Every encoding needs at least one input and output. In Bitmovin parlance an `Input` is an input storage location with a specific transport protocol, for example an HTTPS location. It is _not_ a specific file. Our documentation provides a full list of [supported Inputs](https://bitmovin.com/docs/encoding/articles/supported-input-output-storages).

In [52]:
input = HttpsInput(name=f'{cfg.MY_ID}_LearningLab_Sources',
                   description='Web server for Bitmovin Dev Lab inputs',
                   host="bitmovin-learning-labs-london.s3.amazonaws.com")
input = api.encoding.inputs.https.create(https_input=input)
print("Created input '{}' with id: {}".format(input.name, input.id))

Created input 'fabre_LearningLab_Sources' with id: f42ff8cb-64bf-434f-9cdc-61c5afe85b08


Note how we first create a resource in the SDK, and then submit it to the API for creation. The API will return a full representation of the object, and generated an ID for it. We will use those identifiers to link the various objects that make up the full configuration.

The same concepts apply to the `Output`, which defines where we will store the resulting files.

In [53]:
output = S3Output(name=f'{cfg.MY_ID}_{cfg.S3_BUCKET_NAME}',
                  description='Bucket for Bitmovin Dev Lab outputs',
                  bucket_name=cfg.S3_BUCKET_NAME,
                  access_key=cfg.S3_ACCESS_KEY,
                  secret_key=cfg.S3_SECRET_KEY)
output = api.encoding.outputs.s3.create(s3_output=output)
print("Created output '{}' with id: {}".format(output.name, output.id))

Created output 'fabre_bitmovin-learning-lab-europe' with id: a3bae360-9572-40af-b6c9-176d944d1e6f


_Note_: It is best practice to _reuse_ inputs and outputs you have created before, not create a new one every time. 

You can and should query the API to retrieve the resources you previously created inputs, by their name. The `name` and `description` properties can be added to all Bitmovin resources. 

<img src="img/step2.svg" alt="StreamInput" width="320px" align="right"/>

## Mapping input media streams
We can now define what source file to use in the encoding. 
To do so, we need to create a `StreamInput` resource, which specifies on what _input_ our file is located (by using its ID), at what _path_. We also define what media track to select to decode (and later encode). 

The first input stream we specify is for the video track:

In [54]:
video_input_stream = StreamInput(input_id=input.id, 
                                 input_path="input-files/cosmos_laundromat.mp4", 
                                 selection_mode=StreamSelectionMode.AUTO)

The next one we will create is for the audio. We've specified `StreamSelectionMode.AUDIO_RELATIVE` here and `position=0` to indicate I want the first (0th) audio track in numerical order.

In [55]:
audio_input_stream = StreamInput(input_id=input.id, 
                                 input_path='input-files/cosmos_laundromat.mp4', 
                                 selection_mode=StreamSelectionMode.AUDIO_RELATIVE, 
                                 position=0)

Note that the `StreamInput` is not a resource that is submitted to the API directly. It is an internal object used within the SDK, which is used later in the definition of other resources. 

<img src="img/step3.svg" alt="Configuration" width="320px" align="right"/>

## Configuring the codecs

Next we need to create the codec configurations that define how those files get encoded into the output streams.

We use a helper tuple (a Python-esque construct) to group up our desired output height, bitrate, and video profile.

In [56]:
MyProfile=collections.namedtuple('MyProfile', 'height bitrate profile')

### Ladder

We then define a "ladder" as a set of encoding configurations for the encoder to generate. We will be using H264/AVC in this example.

In [57]:
video_profiles = [
    MyProfile(height=240,  bitrate=400_000,   profile=ProfileH264.MAIN),
    MyProfile(height=360,  bitrate=800_000,   profile=ProfileH264.HIGH),
    MyProfile(height=480,  bitrate=1_200_000, profile=ProfileH264.HIGH),
    MyProfile(height=720,  bitrate=2_400_000, profile=ProfileH264.HIGH),
    MyProfile(height=1080, bitrate=4_800_000, profile=ProfileH264.HIGH),
]

### Video

We can now create each of these video profiles. We will use one of the [preset configurations](https://bitmovin.com/docs/encoding/tutorials/h264-presets), which are templates defined for most common use cases, whether your focus is on performance or quality, and they should always be used.

In [58]:
video_configs = []
for profile in video_profiles:
    video_config = H264VideoConfiguration(
        name=f"{cfg.MY_ID}_H264-{profile.height}p@{profile.bitrate}",
        height=profile.height, 
        bitrate=profile.bitrate, 
        profile=profile.profile,
        preset_configuration=PresetConfiguration.VOD_STANDARD
    )
    video_config = api.encoding.configurations.video.h264.create(video_config)
    video_configs.append(video_config)
    print("Created video codec config '{}' with id: {}".format(video_config.name, video_config.id))

Created video codec config 'fabre_H264-240p@400000' with id: 1347304c-d2c0-4588-a442-668b5736e6d2
Created video codec config 'fabre_H264-360p@800000' with id: 5c9af1a0-26ab-4458-9f3e-6626e69efff5
Created video codec config 'fabre_H264-480p@1200000' with id: 2c05e62e-e000-40e1-aa71-bf02fafb3881
Created video codec config 'fabre_H264-720p@2400000' with id: cb2a3047-5d14-4c12-aa16-347efb3c31eb
Created video codec config 'fabre_H264-1080p@4800000' with id: 530f31e9-f220-4ec7-a737-8327f433fbae


Note that just like inputs and outputs, these resources can and should also be re-used. You can also create them in the dashboard if desired.

### Adaptive bitrate video

Let's pause for a second and cover _why_ we are generating multiple profiles here. 

We are encoding our source video in such a way that it can be played back in a player that supports Adaptive Bitrate (ABR) Streaming. With this mechanism, the Video Player can choose which representation (often called rendition) to play, based on its available bandwidth and capabilities, and can also switch between them dynamically, going to a higher bitrate (and therefore better quality) as the available bandwidth increases, or going to lower bitrates (and lower qualities) as the network conditions deteriorate. 

Each of these representations is a separate encode of the source files, and thus requires a distinct configuration of the encoder.

### Audio
We also need to create the audio configuration. A single AAC stream will do for now.

In [59]:
audio_config = AacAudioConfiguration(
    name=f"{cfg.MY_ID}_AAC-128k",
    bitrate=128_000, 
    rate=48_000.0)
audio_config = api.encoding.configurations.audio.aac.create(aac_audio_configuration=audio_config)
print("Created audio codec config '{}' with id: {}".format(audio_config.name, audio_config.id))

Created audio codec config 'fabre_AAC-128k' with id: 809d41e0-92f5-4045-bad9-560fd3d8b834


## The Encoding itself

<img src="img/step4.svg" alt="Encoding" width="320px" align="right"/>

Each encoding job will have a resource that defines it. We define a number of aspects of the encoding through it:
- The `CloudRegion` defines through what cloud provider and in which region to perform the encoding. We are setting it to `AUTO` here, which means that Bitmovin will attempt to make a "sensible" choice about where to run the encoding. It's best to use a specific region however.

- The `EncoderVersion`: You should set it to `STABLE` to ensure you get the most up to date version of the encoder, for best performance and reliability

In [60]:
encoding = Encoding(name=f"{cfg.MY_ID} - basic encoding tutorial",
                    encoder_version="STABLE",
                    cloud_region=CloudRegion.AUTO)
encoding = api.encoding.encodings.create(encoding=encoding)
print("Created encoding '{}' with id: {}".format(encoding.name, encoding.id))

Created encoding 'fabre - basic encoding tutorial' with id: a11e63e8-df06-4343-9c6c-4f4b0c4b68e8


## Mapping inputs to outputs

<img src="img/step4b.svg" alt="Intermediary Summary" width="320px" align="right"/>

So far we have created:
* An input
* An output
* A set of video and audio "profiles"
* An empty encoding object

Having all these "non-dependent" objects ready, it's now time to connect the chain that will tie in input and output.

### Streams

<img src="img/step5.svg" alt="Streams" width="320px" align="right"/>

We will first create a series of output streams. These simply map one or multiple _input_ streams to a single (elementary) _output_ stream, and are the raw output of the encoding process itself.

For our ABR tutorial use case, there is a simple one-to-one relationship between codecs and video streams.

So, for each config, we create a corresponding `Stream`, which we link to the `StreamInput` created earlier. We link the Stream to a `Configuration`, and attach it to our `Encoding`.

In [61]:
video_streams = []
for config in video_configs:
    stream_shortname = '{}p_{}k'.format(config.height, round(config.bitrate/1000))
    video_stream = Stream(name=f"{cfg.MY_ID}_{stream_shortname}",
                          description=stream_shortname,
                          codec_config_id=config.id,
                          input_streams=[video_input_stream])
    video_stream = api.encoding.encodings.streams.create(encoding.id, stream=video_stream)
    video_streams.append(video_stream)
    print("Created video stream '{}' with id: {}".format(video_stream.name, video_stream.id))

Created video stream 'fabre_240p_400k' with id: 9242f7d5-fb1a-41d7-96b9-7710c8888f1f
Created video stream 'fabre_360p_800k' with id: b8db2ff1-c8d9-41ab-8cee-f930a339a802
Created video stream 'fabre_480p_1200k' with id: c893cb95-4d13-4d08-a3cc-8f61ee92e9d4
Created video stream 'fabre_720p_2400k' with id: 70fd6c1f-7611-4314-8de0-0318339534f2
Created video stream 'fabre_1080p_4800k' with id: 12f79e6e-e918-4fef-b6e2-5de1e881618c


And then we do the same thing for the audio

In [62]:
audio_stream = Stream(name=f'{cfg.MY_ID}_AAC',
                      codec_config_id=audio_config.id, 
                      input_streams=[audio_input_stream])
audio_stream = api.encoding.encodings.streams.create(encoding.id, stream=audio_stream)
print("Created audio stream '{}' with id: {}".format(audio_stream.name, audio_stream.id))

Created audio stream 'fabre_AAC' with id: a3c08184-7f48-4953-88ab-beea155e179b


## Muxing

<img src="img/step6.svg" alt="Muxings" width="320px" align="right"/>

Raw output isn't enough however. An output stream must be _muxed_ into a container, for example an MPEG Transport Stream, or fragmented MPEG 4 boxes (ISOBMFF). 

For each item we need to add our stream to a `MuxingStream`, which takes a stream, an `EncodingOutput` which specifies the output and the _path_ for this muxing and then create the `Muxing` itself. 

Note that muxings may contain multiple streams, and be replicated to multiple outputs. For simplicity's sake here, and in line with standard ABR practices, we create a separate muxing for each generated track.

In [63]:
video_muxings = []
for video_stream in video_streams:
    muxing_stream = MuxingStream(stream_id=video_stream.id)
    muxing_output = EncodingOutput(output_id=output.id,
                                   output_path='{}/video/{}'.format(base_output_path, video_stream.description),
                                   acl=[AclEntry(permission=AclPermission.PUBLIC_READ)])
    video_muxing = Fmp4Muxing(name=video_stream.name + "_fmp4",
                              streams=[muxing_stream],
                              segment_length=4.0,
                              segment_naming="seg_%number%.m4s",
                              init_segment_name='init.mp4',
                              outputs=[muxing_output])
    video_muxing = api.encoding.encodings.muxings.fmp4.create(encoding_id=encoding.id, fmp4_muxing=video_muxing)
    video_muxings.append(video_muxing)
    print("Created video muxing '{}' with id: {}".format(video_muxing.name, video_muxing.id))

Created video muxing 'fabre_240p_400k_fmp4' with id: 5b0c34fb-fc70-4e65-9e6a-4eb1b0488590
Created video muxing 'fabre_360p_800k_fmp4' with id: 56cc5103-67c8-44c6-b7ac-b033020a42c8
Created video muxing 'fabre_480p_1200k_fmp4' with id: 25528308-934c-48bc-af1c-e33f5eb73a9a
Created video muxing 'fabre_720p_2400k_fmp4' with id: b0cb3353-2827-4f8d-bdcc-8ea8e6f7ab0c
Created video muxing 'fabre_1080p_4800k_fmp4' with id: d44de333-5eff-4e4b-af26-b5b7d4eec0a9


We also create a separate audio muxing, into a separate folder. We could mux the audio with our video streams, but standard practice with ABR is to have separate audio-only muxings.


In [64]:
audio_muxing_stream = MuxingStream(stream_id=audio_stream.id)
audio_muxing_output = EncodingOutput(output_id=output.id,
                                     output_path=base_output_path+'/audio/',
                                     acl=[AclEntry(scope='*', permission=AclPermission.PUBLIC_READ)])
audio_muxing = Fmp4Muxing(name=f"{audio_stream.name}_fmp4",
                          streams=[audio_muxing_stream],
                          segment_length=4.0,
                          segment_naming="seg_%number%.m4s",
                          init_segment_name='init.mp4',
                          outputs=[audio_muxing_output])
audio_muxing = api.encoding.encodings.muxings.fmp4.create(encoding_id=encoding.id, fmp4_muxing=audio_muxing)
print("Created audio muxing '{}' with id: {}".format(audio_muxing.name, audio_muxing.id))

Created audio muxing 'fabre_AAC_fmp4' with id: c8c5fadc-68f2-4db8-84e6-d623718fb75c


<img src="img/step7.svg" alt="Start" width="320px" align="right"/>

## Starting the encoding...

Next we are going to start the encode! 

In [65]:
api.encoding.encodings.start(encoding.id)
print("Starting encoding")

Starting encoding


{'id': 'a11e63e8-df06-4343-9c6c-4f4b0c4b68e8'}

In [67]:
url = helpers.build_dashboard_url(encoding.id)
display(
    p("You can now check encoding progress in the dashboard at ", 
      a(f"{url}", href=f"{url}", target="_new")
    )
)    

<p>You can now check encoding progress in the dashboard at <a href="https://bitmovin.com/dashboard/encoding/encodings/a11e63e8-df06-4343-9c6c-4f4b0c4b68e8?apiKey=2d05c68d-1dab-4efb-bb3f-a849d17d71cd&amp;orgId=6f754aa8-fd27-4102-b539-52f1f9c26818" target="_new">https://bitmovin.com/dashboard/encoding/encodings/a11e63e8-df06-4343-9c6c-4f4b0c4b68e8?apiKey=2d05c68d-1dab-4efb-bb3f-a849d17d71cd&amp;orgId=6f754aa8-fd27-4102-b539-52f1f9c26818</a></p>

<img src="img/step8.svg" alt="Monitoring" width="320px" align="right"/>

### ... and monitoring it

You can monitor the encoding in your script by polling its status on a regular basis. This is the easiest way to keep track of the encoding when you are testing your encoding configuration.

For production environments however, you should use [webhooks](//https://bitmovin.com/docs/encoding/api-reference/sections/notifications-webhooks) instead.

In [68]:
while True:
    task = api.encoding.encodings.status(encoding.id)
    print("Got task status {} - {}%".format(task.status, task.progress))
    if task.status == Status.ERROR:
        print("Error during encoding!")
        raise SystemExit
    if task.status == Status.FINISHED:
        print("Encoding complete")
        break
    time.sleep(15)

Got task status Status.QUEUED - 0%
Got task status Status.QUEUED - 0%
Got task status Status.QUEUED - 0%
Got task status Status.QUEUED - 0%
Got task status Status.RUNNING - 5%
Got task status Status.RUNNING - 18%
Got task status Status.RUNNING - 40%
Got task status Status.RUNNING - 40%
Got task status Status.RUNNING - 41%
Got task status Status.RUNNING - 41%
Got task status Status.RUNNING - 46%
Got task status Status.RUNNING - 50%
Got task status Status.RUNNING - 57%
Got task status Status.RUNNING - 61%
Got task status Status.RUNNING - 68%
Got task status Status.RUNNING - 74%
Got task status Status.FINISHED - 100%
Encoding complete


## Combining into a manifest

<img src="img/step9.svg" alt="Manifest" width="320px" align="right"/>

We will ask the encoder to generate the `Manifest` as well. The manifest is used by ABR players to find all information about quality levels, audio tracks, subtitles etc. 

We are going to generate a _DASH_ (Dynamic Adaptive Streaming over HTTP) manifest, which can be played on Android and iOS devices, as well as the Bitmovin player on most platforms.

Bitmovin provides you with full flexibility to create manifests in a fine grained way. But we will be using `DefaultManifest` functionality, which will apply smart defaults to create a standard manifest. 

In [69]:
print("Creating manifests")
manifest_output = EncodingOutput(output_id=output.id,
                                 output_path=base_output_path+'/',
                                 acl=[AclEntry(scope='*', permission=AclPermission.PUBLIC_READ)])
dash_manifest = DashManifestDefault(
    name=f"{cfg.MY_ID}_DashManifest",
    manifest_name="stream.mpd",
    encoding_id=encoding.id,
    version=DashManifestDefaultVersion.V1,
    outputs=[manifest_output])

dash_manifest = api.encoding.manifests.dash.default.create(dash_manifest)
print("Created manifest '{}' with id: {}".format(dash_manifest.name, dash_manifest.id))

Creating manifests
Created manifest 'fabre_DashManifest' with id: f28c38bc-bbe1-4b35-81a6-e6baad8ee41f


And now we can trigger the generation of the manifest... 

In [70]:
api.encoding.manifests.dash.start(dash_manifest.id)
print("Generating manifest")

{'id': 'f28c38bc-bbe1-4b35-81a6-e6baad8ee41f'}

... and monitor it

In [77]:
while True:
    time.sleep(5)
    status = api.encoding.manifests.dash.status(dash_manifest.id).status
    if status == Status.FINISHED:
        break
    if status == Status.ERROR:
        print("Error during dash manifest generation")
        raise SystemExit

manifest_url = "https://"+cfg.S3_BUCKET_NAME+".s3.amazonaws.com/" + manifest_output.output_path + dash_manifest.manifest_name
display( p(f"Manifest URL: ", b(f"{manifest_url}")) )

<p>Manifest URL: <b>https://bitmovin-learning-lab-europe.s3.amazonaws.com/outputs/fabre-49bdbc9d-c8fb-4ce7-881c-d5df402028a9/stream.mpd</b></p>

# Playback test
You can now try playing back the stream, by using the manifest URL in our test player at https://bitmovin.com/demos/stream-test

## In your own player
Alternatively, you can just play it right here. 
You may have to whitelist the "google.com" domain for your player license first.

To retrieve your license and add the domain, head to the Dashboard at https://bitmovin.com/dashboard/player/licenses


In [72]:
cfg.PLAYER_LICENSE='f9e2cf25-9cdd-4c9d-a314-90fdb6d5590c'

In [75]:
embed_url = "https://demo.bitmovin.com/public/learning-labs/encoding/test-players/basic-dash-player.html?"
embed_url += "license="+cfg.PLAYER_LICENSE
embed_url += "&mpdurl="+manifest_url

IFrame(src=embed_url, width=800, height=450)